In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pcamv1/camelyonpatch_level_2_split_train_y.h5
/kaggle/input/pcamv1/camelyonpatch_level_2_split_valid_y.h5
/kaggle/input/pcamv1/camelyonpatch_level_2_split_valid_meta.csv
/kaggle/input/pcamv1/camelyonpatch_level_2_split_valid_x.h5
/kaggle/input/pcamv1/camelyonpatch_level_2_split_train_mask.h5
/kaggle/input/pcamv1/camelyonpatch_level_2_split_train_meta.csv
/kaggle/input/pcamv1/camelyonpatch_level_2_split_test_y.h5
/kaggle/input/pcamv1/camelyonpatch_level_2_split_test_meta.csv
/kaggle/input/pcamv1/camelyonpatch_level_2_split_test_x.h5
/kaggle/input/pcamv1/camelyonpatch_level_2_split_train_x.h5-001/camelyonpatch_level_2_split_train_x.h5


In [3]:
# Core
import os
import time
import random
import yaml
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)


In [4]:
import yaml

config = {
    "seed": 1337,
    "n_seeds": 3,
    "device": "cuda:0",
    "deterministic": True,

    "data": {
        "dataset_version": "pcam_v1",
        "splits_file": "splits/patient_level_fold0.json",
        "img_size": 128,
        "normalize": {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]},
        "decode": "pillow_rgb",
        "stain_norm": "none",
        "augment": {
            "train": [
                {"hflip": 0.5},
                {"vflip": 0.5},
                {"color_jitter": {"b": 0.2, "c": 0.2, "s": 0.2}},
                {"blur": {"p": 0.2, "k": 3}},
            ],
            "val": []
        },
        "sampler": "random",
        "samples_per_epoch": None,
    },

    "loader": {
        "batch_size": 64,
        "grad_accum": 1,
        "num_workers": 4,
        "pin_memory": True,
        "persistent_workers": True,
    },

    "model": {
        "backbone": "densenet121",
        "weights": "imagenet_v1",
        "precision": "fp32",
        "dropout": 0.0,
        "freeze_backbone": False,
    },

    "objective": {
        "loss": "bce_logits",
        "pos_weight": None,
        "label_smoothing": 0.0,
        "focal": {"enable": False, "alpha": 0.25, "gamma": 2.0},
        "clip_grad_norm": None,
        "calibration": {"enable": False},
        "decision_threshold": 0.5,
        "tta": {"enable": False},
    },

    "optim": {
        "name": "adamw",
        "lr": 3.0e-4,
        "weight_decay": 1.0e-4,
        "betas": [0.9, 0.999],
        "schedule": "cosine",
        "t_max": 30,
        "warmup_steps": 0,
        "epochs": 10,
        "ema": {"enable": False, "decay": 0.999},
    },

    "eval": {
        "metrics": ["roc_auc", "pr_auc", "acc", "f1"],
        "ci_bootstrap": 2000,
        "select_best_by": "roc_auc",
        "eval_every": "epoch",
    },

    "env": {
        "torch": "2.3.1",
        "torchvision": "0.18.1",
        "cuda": "12.1",
        "cudnn_deterministic": True,
        "cublas_workspace": ":4096:8",
    }
}

# Save to YAML
with open("experiment.yaml", "w") as f:
    yaml.dump(config, f, sort_keys=False)

print("✅ experiment.yaml file created")

✅ experiment.yaml file created


In [5]:
import random, numpy as np, torch

with open("experiment.yaml", "r") as f:
    cfg = yaml.safe_load(f)

# -----------------------------
# Seed & Determinism
# -----------------------------
def set_seed(seed: int, deterministic: bool = True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    else:
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True

set_seed(cfg["seed"], cfg["deterministic"])

# -----------------------------
# Device
# -----------------------------
device = torch.device(cfg["device"] if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda:0


In [7]:
from torch.utils.data import Dataset
# --------------------------------
# Chunked Dataset for Memory Efficiency
# --------------------------------
class PCamDatasetChunk(Dataset):
    def __init__(self, h5_path, labels_path=None, transform=None, start_idx=0, end_idx=None):
        """
        h5_path: path to HDF5 file with images
        labels_path: path to HDF5 file with labels
        transform: torchvision transforms
        start_idx, end_idx: slice of the dataset to load (for splitting)
        """
        self.h5_path = h5_path
        self.labels_path = labels_path
        self.transform = transform
        
        # Open HDF5 files and slice the dataset
        self.h5_file = h5py.File(h5_path, "r")
        key = "x" if "x" in self.h5_file else list(self.h5_file.keys())[0]
        
        if end_idx is None:
            end_idx = len(self.h5_file[key])
            
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.length = end_idx - start_idx
        
        # Memory-map the selected chunk
        self.images = self.h5_file[key][start_idx:end_idx]
        
        # Load labels
        self.labels = None
        if labels_path is not None:
            with h5py.File(labels_path, "r") as f:
                self.labels = f["y"][start_idx:end_idx]
        
        print(f"Dataset chunk initialized: samples {start_idx} to {end_idx} ({self.length} total)")

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        img = self.images[idx]
        img = Image.fromarray(img)
        
        if self.transform:
            img = self.transform(img)
            
        if self.labels is not None:
            label = int(self.labels[idx].item())
            return img, label
        else:
            return img

    def close(self):
        if hasattr(self, 'h5_file') and self.h5_file:
            self.h5_file.close()


In [18]:
# --------------------------------
# Transforms from Config
# --------------------------------
cfg_data = cfg["data"]

train_transform = transforms.Compose([
    transforms.Resize((cfg_data["img_size"], cfg_data["img_size"])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=cfg_data["normalize"]["mean"], 
                        std=cfg_data["normalize"]["std"])
])

val_transform = transforms.Compose([
    transforms.Resize((cfg_data["img_size"], cfg_data["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=cfg_data["normalize"]["mean"], 
                        std=cfg_data["normalize"]["std"])
])

# --------------------------------
# Dataset Paths
# --------------------------------
h5_train_x = "/kaggle/input/pcamv1/camelyonpatch_level_2_split_train_x.h5-001/camelyonpatch_level_2_split_train_x.h5"
h5_train_y = "/kaggle/input/pcamv1/camelyonpatch_level_2_split_train_y.h5"
h5_val_x   = "/kaggle/input/pcamv1/camelyonpatch_level_2_split_valid_x.h5"
h5_val_y   = "/kaggle/input/pcamv1/camelyonpatch_level_2_split_valid_y.h5"

# Get total training samples
with h5py.File(h5_train_x, "r") as f:
    total_train = len(f["x"])
    print(f"Total training samples: {total_train}")

# Split training data into chunks to manage memory
num_chunks = 4  # Split into 4 chunks (~65K samples each)
chunk_size = total_train // num_chunks
train_splits = []

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < num_chunks - 1 else total_train
    train_splits.append((start_idx, end_idx))

print(f"Training data split into {num_chunks} chunks:")
for i, (start, end) in enumerate(train_splits):
    print(f"  Chunk {i+1}: samples {start} to {end} ({end-start} samples)")


Total training samples: 262144
Training data split into 4 chunks:
  Chunk 1: samples 0 to 65536 (65536 samples)
  Chunk 2: samples 65536 to 131072 (65536 samples)
  Chunk 3: samples 131072 to 196608 (65536 samples)
  Chunk 4: samples 196608 to 262144 (65536 samples)


In [19]:
# --------------------------------
# Create Validation Dataset
# --------------------------------
print("\nCreating validation dataset...")
val_dataset = PCamDatasetChunk(h5_val_x, labels_path=h5_val_y, transform=val_transform)

cfg_loader = cfg["loader"]
val_loader = DataLoader(
    val_dataset, 
    batch_size=cfg_loader["batch_size"], 
    shuffle=False, 
    num_workers=cfg_loader["num_workers"], 
    pin_memory=cfg_loader["pin_memory"]
)

print(f"Validation dataset created: {len(val_dataset)} samples, {len(val_loader)} batches")


Creating validation dataset...
Dataset chunk initialized: samples 0 to 32768 (32768 total)
Validation dataset created: 32768 samples, 512 batches


In [20]:
# --------------------------------
# Model Setup
# --------------------------------
cfg_model = cfg["model"]
print("Creating model...")

class DNBCD(nn.Module):
    def __init__(self, dropout1=0.3, dropout2=0.3):
        super(DNBCD, self).__init__()
        if cfg_model["weights"] == "imagenet_v1":
            self.backbone = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        else:
            self.backbone = models.densenet121(weights=None)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-1])  # Remove classifier
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.dropout1 = nn.Dropout(dropout1)
        self.fc1 = nn.Linear(1024, 256)
        self.bn = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout2)
        self.fc2 = nn.Linear(256, 1)  # Output raw logits

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

if cfg_model["backbone"] == "densenet121":
    model = DNBCD().to(device)  # Use defaults or config values if added to YAML
else:
    raise NotImplementedError(f"Backbone {cfg_model['backbone']} not implemented")

if cfg_model["freeze_backbone"]:
    for param in model.backbone.parameters():
        param.requires_grad = False
    print("Backbone frozen")

# Count and display parameters in a table
try:
    from tabulate import tabulate
    table_available = True
except ImportError:
    table_available = False
    print("Note: tabulate library not available, using plain print for parameter table")

print("\nModel Parameter Breakdown:")
param_table = []
total_params = 0
trainable_params = 0

for name, param in model.named_parameters():
    param_count = param.numel()
    total_params += param_count
    if param.requires_grad:
        trainable_params += param_count
    param_table.append([name, str(tuple(param.shape)), f"{param_count:,}"])

if table_available:
    headers = ["Layer Name", "Parameter Shape", "Number of Parameters"]
    print(tabulate(param_table, headers=headers, tablefmt="grid"))
else:
    print("Layer Name | Parameter Shape | Number of Parameters")
    print("-" * 60)
    for row in param_table:
        print(f"{row[0]:<40} | {row[1]:<15} | {row[2]:>10}")

print(f"\n✅ Model created - Total params: {total_params:,}, Trainable: {trainable_params:,}")

Creating model...


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 103MB/s] 



Model Parameter Breakdown:
+--------------------------------------------------+-------------------+------------------------+
| Layer Name                                       | Parameter Shape   | Number of Parameters   |
+==================================================+===================+========================+
| backbone.0.conv0.weight                          | (64, 3, 7, 7)     | 9,408                  |
+--------------------------------------------------+-------------------+------------------------+
| backbone.0.norm0.weight                          | (64,)             | 64                     |
+--------------------------------------------------+-------------------+------------------------+
| backbone.0.norm0.bias                            | (64,)             | 64                     |
+--------------------------------------------------+-------------------+------------------------+
| backbone.0.denseblock1.denselayer1.norm1.weight  | (64,)             | 64               

In [21]:
# --------------------------------
# Training Setup
# --------------------------------
cfg_objective = cfg["objective"]
cfg_optim = cfg["optim"]

criterion = nn.BCEWithLogitsLoss()
decision_threshold = cfg_objective["decision_threshold"]

optimizer = optim.AdamW(
    model.parameters(),
    lr=cfg_optim["lr"],
    weight_decay=cfg_optim["weight_decay"],
    betas=tuple(cfg_optim["betas"])
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=cfg_optim["t_max"]
) if cfg_optim["schedule"].lower() == "cosine" else None

print("✅ Training setup complete")

✅ Training setup complete


In [22]:
import gc
# --------------------------------
# Training Loop
# --------------------------------
epochs = cfg_optim["epochs"]
train_losses = []
val_losses = []
roc_aucs = []
pr_aucs = []
accs = []
f1s = []

print(f"\n{'='*60}")
print("STARTING CHUNKED TRAINING")
print(f"{'='*60}")
print(f"Epochs: {epochs}, Chunks per epoch: {num_chunks}")

total_start_time = time.time()

# Add this before: for epoch in range(epochs):
best_roc_auc = 0.0
checkpoint_path = "densenet121_chunked_best.pth"  # Or add to YAML as cfg["eval"]["checkpoint_path"]

for epoch in range(epochs):
    epoch_start_time = time.time()
    print(f"\n{'='*50}")
    print(f"EPOCH {epoch+1}/{epochs}")
    print(f"{'='*50}")
    
    # Training phase - iterate through chunks
    model.train()
    epoch_train_loss = 0.0
    epoch_train_samples = 0
    
    for chunk_idx, (start_idx, end_idx) in enumerate(train_splits):
        chunk_start_time = time.time()
        print(f"\nTraining on chunk {chunk_idx+1}/{num_chunks}: samples {start_idx} to {end_idx}")
        
        # Create dataset for this chunk
        train_dataset = PCamDatasetChunk(
            h5_train_x, 
            labels_path=h5_train_y,
            transform=train_transform,
            start_idx=start_idx,
            end_idx=end_idx
        )
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=cfg_loader["batch_size"],
            shuffle=True, 
            num_workers=cfg_loader["num_workers"], 
            pin_memory=cfg_loader["pin_memory"]
        )
        
        # Train on this chunk
        chunk_loss = 0.0
        chunk_samples = 0
        
        for step, (images, labels) in enumerate(train_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            chunk_loss += loss.item() * images.size(0)
            chunk_samples += images.size(0)
            
            if step % 100 == 0:
                current_loss = chunk_loss / chunk_samples
                print(f"  Step {step:4d}/{len(train_loader)} - Loss: {current_loss:.4f}")
        
        # Close the chunk dataset
        train_dataset.close()
        del train_dataset, train_loader
        gc.collect()
        
        chunk_time = time.time() - chunk_start_time
        chunk_avg_loss = chunk_loss / chunk_samples
        samples_per_sec = chunk_samples / chunk_time
        
        print(f"  Chunk {chunk_idx+1} completed in {chunk_time:.1f}s - "
              f"Loss: {chunk_avg_loss:.4f} - Speed: {samples_per_sec:.0f} samples/sec")
        
        epoch_train_loss += chunk_loss
        epoch_train_samples += chunk_samples
    
    # Calculate epoch training loss
    epoch_train_loss /= epoch_train_samples
    
    # Validation phase
    print(f"\nValidation...")
    val_start_time = time.time()
    model.eval()
    val_loss = 0.0
    val_samples = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(val_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            val_samples += images.size(0)
            
            # Collect predictions for metrics
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.cpu().numpy())
    
    val_time = time.time() - val_start_time
    val_loss /= val_samples
    
    # Compute metrics
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    roc_auc = roc_auc_score(all_labels, all_preds)
    pr_auc = average_precision_score(all_labels, all_preds)
    acc = accuracy_score(all_labels, (all_preds > decision_threshold).astype(int))
    f1 = f1_score(all_labels, (all_preds > decision_threshold).astype(int))
    
    # Store metrics
    train_losses.append(epoch_train_loss)
    val_losses.append(val_loss)
    roc_aucs.append(roc_auc)
    pr_aucs.append(pr_auc)
    accs.append(acc)
    f1s.append(f1)


    if roc_auc > best_roc_auc:
       best_roc_auc = roc_auc
       torch.save(model.state_dict(), checkpoint_path)
       print(f"✅ Saved best model at epoch {epoch+1} with ROC-AUC: {best_roc_auc:.4f}")
    
    # Epoch summary
    epoch_time = time.time() - epoch_start_time
    
    print(f"\n{'='*60}")
    print(f"EPOCH {epoch+1} COMPLETE ({epoch_time:.1f}s total)")
    print(f"{'='*60}")
    print(f"Training Loss  : {epoch_train_loss:.4f}")
    print(f"Validation     : {val_time:.1f}s - Loss: {val_loss:.4f}")
    print(f"Metrics        : ROC-AUC: {roc_auc:.4f} | PR-AUC: {pr_auc:.4f} | Acc: {acc:.4f} | F1: {f1:.4f}")
    
    if scheduler:
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step()
        new_lr = optimizer.param_groups[0]['lr']
        print(f"Learning Rate  : {old_lr:.6f} -> {new_lr:.6f}")
    
    # Clean up validation arrays
    del all_preds, all_labels
    gc.collect()

# Training complete
total_time = time.time() - total_start_time
avg_epoch_time = total_time / epochs

print(f"\n{'='*60}")
print("TRAINING COMPLETED!")
print(f"{'='*60}")
print(f"Total time: {total_time/60:.1f} minutes")
print(f"Average epoch time: {avg_epoch_time:.1f} seconds")
print(f"Best ROC-AUC: {max(roc_aucs):.4f} (Epoch {roc_aucs.index(max(roc_aucs))+1})")

# Save results
torch.save(model.state_dict(), "densenet121_chunked.pth")
print("✅ Model saved to densenet121_chunked.pth")


STARTING CHUNKED TRAINING
Epochs: 10, Chunks per epoch: 4

EPOCH 1/10

Training on chunk 1/4: samples 0 to 65536
Dataset chunk initialized: samples 0 to 65536 (65536 total)
  Step    0/1024 - Loss: 0.7348
  Step  100/1024 - Loss: 0.3211
  Step  200/1024 - Loss: 0.2898
  Step  300/1024 - Loss: 0.2647
  Step  400/1024 - Loss: 0.2557
  Step  500/1024 - Loss: 0.2451
  Step  600/1024 - Loss: 0.2371
  Step  700/1024 - Loss: 0.2304
  Step  800/1024 - Loss: 0.2250
  Step  900/1024 - Loss: 0.2198
  Step 1000/1024 - Loss: 0.2149
  Chunk 1 completed in 144.2s - Loss: 0.2139 - Speed: 454 samples/sec

Training on chunk 2/4: samples 65536 to 131072
Dataset chunk initialized: samples 65536 to 131072 (65536 total)
  Step    0/1024 - Loss: 0.1776
  Step  100/1024 - Loss: 0.1625
  Step  200/1024 - Loss: 0.1653
  Step  300/1024 - Loss: 0.1676
  Step  400/1024 - Loss: 0.1665
  Step  500/1024 - Loss: 0.1651
  Step  600/1024 - Loss: 0.1630
  Step  700/1024 - Loss: 0.1607
  Step  800/1024 - Loss: 0.1599
  S

In [40]:
import pandas as pd
training_logs = pd.DataFrame({
    "epoch": list(range(1, epochs+1)),
    "train_loss": train_losses,
    "val_loss": val_losses,
    "roc_auc": roc_aucs,
    "pr_auc": pr_aucs,
    "accuracy": accs,
    "f1": f1s
})
training_logs.to_csv("training_logs_chunked.csv", index=False)
print("✅ Training logs saved to training_logs_chunked.csv")

# Clean up validation dataset
val_dataset.close()
print("✅ Training complete - all resources cleaned up!")

✅ Training logs saved to training_logs_chunked.csv
✅ Training complete - all resources cleaned up!


In [41]:
# -----------------------------
# Load Trained Model
# -----------------------------
cfg_model = cfg["model"]

# Recreate model architecture
if cfg_model["backbone"] == "densenet121":
    model = models.densenet121(weights=None)  # No pretrained weights needed for inference
    model.classifier = nn.Sequential(
        nn.Dropout(cfg_model["dropout"]),
        nn.Linear(model.classifier.in_features, 1)
    )
else:
    raise NotImplementedError(f"Backbone {cfg_model['backbone']} not implemented")

# Load trained weights
model_path = "densenet121_chunked_best.pth"  
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"✅ Model loaded from {model_path}")
except FileNotFoundError:
    print(f"❌ Model file {model_path} not found. Please train the model first.")
    exit()

model = model.to(device)
model.eval()

# -----------------------------
# Test Dataset
# -----------------------------
from torchvision import transforms
from PIL import Image

cfg_data = cfg["data"]

test_transform = transforms.Compose([
    transforms.Resize((cfg_data["img_size"], cfg_data["img_size"])),
    transforms.ToTensor(),
    transforms.Normalize(mean=cfg_data["normalize"]["mean"], 
                        std=cfg_data["normalize"]["std"])
])

class PCamDatasetChunk(Dataset):
    def __init__(self, h5_path, labels_path=None, transform=None, start_idx=0, end_idx=None):
        self.h5_path = h5_path
        self.labels_path = labels_path
        self.transform = transform
        
        self.h5_file = h5py.File(h5_path, "r")
        key = "x" if "x" in self.h5_file else list(self.h5_file.keys())[0]
        
        if end_idx is None:
            end_idx = len(self.h5_file[key])
            
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.length = end_idx - start_idx
        
        self.images = self.h5_file[key][start_idx:end_idx]
        
        self.labels = None
        if labels_path is not None:
            with h5py.File(labels_path, "r") as f:
                self.labels = f["y"][start_idx:end_idx]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        img = self.images[idx]
        img = Image.fromarray(img)
        
        if self.transform:
            img = self.transform(img)
            
        if self.labels is not None:
            label = int(self.labels[idx].item())
            return img, label
        else:
            return img

    def close(self):
        if hasattr(self, 'h5_file') and self.h5_file:
            self.h5_file.close()

# Test dataset paths
h5_test_x = "/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_test_x.h5"
h5_test_y = "/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_test_y.h5"

print("Creating test dataset...")
test_dataset = PCamDatasetChunk(h5_test_x, labels_path=h5_test_y, transform=test_transform)

cfg_loader = cfg["loader"]
test_loader = DataLoader(
    test_dataset, 
    batch_size=cfg_loader["batch_size"], 
    shuffle=False,  # Don't shuffle test data
    num_workers=cfg_loader["num_workers"], 
    pin_memory=cfg_loader["pin_memory"]
)

print(f"Test dataset: {len(test_dataset)} samples, {len(test_loader)} batches")

# -----------------------------
# Test Evaluation
# -----------------------------
print(f"\n{'='*50}")
print("EVALUATING ON TEST SET")
print(f"{'='*50}")

decision_threshold = cfg["objective"]["decision_threshold"]

start_time = time.time()
all_preds = []
all_labels = []
all_probabilities = []
test_loss = 0.0
criterion = nn.BCEWithLogitsLoss()

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(test_loader):
        images = images.to(device, non_blocking=True)
        labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
        
        # Forward pass
        outputs = model(images)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        
        # Get predictions and probabilities
        probabilities = torch.sigmoid(outputs).cpu().numpy()
        predictions = (probabilities > decision_threshold).astype(int)
        
        all_probabilities.append(probabilities)
        all_preds.append(predictions)
        all_labels.append(labels.cpu().numpy())
        
        if batch_idx % 100 == 0:
            print(f"  Processed batch {batch_idx}/{len(test_loader)}")

evaluation_time = time.time() - start_time
test_loss /= len(test_dataset)

# Combine all predictions
all_probabilities = np.vstack(all_probabilities)
all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

print(f"✅ Evaluation completed in {evaluation_time:.1f} seconds")

# -----------------------------
# Calculate Metrics
# -----------------------------
print(f"\n{'='*50}")
print("TEST RESULTS")
print(f"{'='*50}")

# Core metrics
roc_auc = roc_auc_score(all_labels, all_probabilities)
pr_auc = average_precision_score(all_labels, all_probabilities)
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Test Loss      : {test_loss:.4f}")
print(f"ROC-AUC        : {roc_auc:.4f}")
print(f"PR-AUC         : {pr_auc:.4f}")
print(f"Accuracy       : {accuracy:.4f}")
print(f"F1-Score       : {f1:.4f}")

# Detailed classification report
print(f"\n{'='*30}")
print("CLASSIFICATION REPORT")
print(f"{'='*30}")
class_names = ['Normal', 'Tumor']
print(classification_report(all_labels.flatten(), all_preds.flatten(), 
                          target_names=class_names, digits=4))

# Confusion Matrix
cm = confusion_matrix(all_labels.flatten(), all_preds.flatten())
print(f"\nConfusion Matrix:")
print(f"{'':>8} {'Predicted':>18}")
print(f"{'':>8} {'Normal':>8} {'Tumor':>8}")
print(f"{'Normal':>8} {cm[0,0]:>8} {cm[0,1]:>8}")
print(f"{'Tumor':>8} {cm[1,0]:>8} {cm[1,1]:>8}")

# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)  # Recall for positive class
specificity = tn / (tn + fp)  # Recall for negative class
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative predictive value

print(f"\nDetailed Metrics:")
print(f"Sensitivity (Recall)    : {sensitivity:.4f}")
print(f"Specificity             : {specificity:.4f}")
print(f"Precision (PPV)         : {precision:.4f}")
print(f"Negative Pred. Value    : {npv:.4f}")

# -----------------------------
# Save Results
# -----------------------------
print(f"\n{'='*50}")
print("SAVING RESULTS")
print(f"{'='*50}")

# Save detailed results
test_results = {
    'test_loss': test_loss,
    'roc_auc': roc_auc,
    'pr_auc': pr_auc,
    'accuracy': accuracy,
    'f1_score': f1,
    'sensitivity': sensitivity,
    'specificity': specificity,
    'precision': precision,
    'negative_predictive_value': npv,
    'true_negatives': int(tn),
    'false_positives': int(fp),
    'false_negatives': int(fn),
    'true_positives': int(tp),
    'decision_threshold': decision_threshold,
    'evaluation_time_seconds': evaluation_time
}

# Save as JSON
import json
with open('test_results.json', 'w') as f:
    json.dump(test_results, f, indent=2)

# Save predictions
predictions_df = pd.DataFrame({
    'sample_id': range(len(all_labels)),
    'true_label': all_labels.flatten(),
    'probability': all_probabilities.flatten(),
    'prediction': all_preds.flatten(),
    'correct': (all_labels.flatten() == all_preds.flatten()).astype(int)
})
predictions_df.to_csv('test_predictions.csv', index=False)

print("✅ Results saved to:")
print("  - test_results.json (summary metrics)")
print("  - test_predictions.csv (individual predictions)")

# -----------------------------
# Optional: Create Visualizations
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Set up plotting style
    plt.style.use('default')
    fig, axes = plt.subplots(3, 2, figsize=(15, 18))  # Expanded for new plots
    
    # 1. ROC Curve
    from sklearn.metrics import roc_curve
    fpr, tpr, _ = roc_curve(all_labels, all_probabilities)
    
    axes[0,0].plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})', linewidth=2)
    axes[0,0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
    axes[0,0].set_xlabel('False Positive Rate')
    axes[0,0].set_ylabel('True Positive Rate')
    axes[0,0].set_title('ROC Curve')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Precision-Recall Curve
    from sklearn.metrics import precision_recall_curve
    precision_curve, recall_curve, _ = precision_recall_curve(all_labels, all_probabilities)
    
    axes[0,1].plot(recall_curve, precision_curve, label=f'PR Curve (AUC = {pr_auc:.4f})', linewidth=2)
    axes[0,1].set_xlabel('Recall')
    axes[0,1].set_ylabel('Precision')
    axes[0,1].set_title('Precision-Recall Curve')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Confusion Matrix Heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names, ax=axes[1,0])
    axes[1,0].set_title('Confusion Matrix')
    axes[1,0].set_ylabel('True Label')
    axes[1,0].set_xlabel('Predicted Label')
    
    # 4. Prediction Distribution
    axes[1,1].hist(all_probabilities[all_labels.flatten() == 0], bins=50, alpha=0.7, 
                  label='Normal', color='blue', density=True)
    axes[1,1].hist(all_probabilities[all_labels.flatten() == 1], bins=50, alpha=0.7, 
                  label='Tumor', color='red', density=True)
    axes[1,1].axvline(decision_threshold, color='black', linestyle='--', 
                     label=f'Threshold ({decision_threshold})')
    axes[1,1].set_xlabel('Prediction Probability')
    axes[1,1].set_ylabel('Density')
    axes[1,1].set_title('Prediction Distribution')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # 5. NEW: Training/Validation Loss Curves
    axes[2,0].plot(train_losses, label='Train Loss', color='blue', linewidth=2)
    axes[2,0].plot(val_losses, label='Val Loss', color='orange', linewidth=2)
    axes[2,0].set_xlabel('Epoch')
    axes[2,0].set_ylabel('Loss')
    axes[2,0].set_title('Training and Validation Loss')
    axes[2,0].legend()
    axes[2,0].grid(True, alpha=0.3)
    
    # 6. NEW: Training/Validation Accuracy Curves (using accs from original)
    axes[2,1].plot(accs, label='Val Accuracy', color='green', linewidth=2)  # Original stores val acc in accs
    axes[2,1].set_xlabel('Epoch')
    axes[2,1].set_ylabel('Accuracy')
    axes[2,1].set_title('Validation Accuracy per Epoch')
    axes[2,1].legend()
    axes[2,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('test_evaluation_plots.png', dpi=300, bbox_inches='tight')
    print("  - test_evaluation_plots.png (visualization)")
    
except ImportError:
    print("Note: matplotlib/seaborn not available for plotting")
  

# -----------------------------
# Cleanup
# -----------------------------
test_dataset.close()
print("\n✅ Test evaluation complete!")

# Print final summary
print(f"\n{'='*60}")
print("FINAL TEST SUMMARY")
print(f"{'='*60}")
print(f"Dataset: {len(test_dataset)} samples")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Evaluation time: {evaluation_time:.1f} seconds")
print(f"{'='*60}")

RuntimeError: Error(s) in loading state_dict for DenseNet:
	Missing key(s) in state_dict: "features.conv0.weight", "features.norm0.weight", "features.norm0.bias", "features.norm0.running_mean", "features.norm0.running_var", "features.denseblock1.denselayer1.norm1.weight", "features.denseblock1.denselayer1.norm1.bias", "features.denseblock1.denselayer1.norm1.running_mean", "features.denseblock1.denselayer1.norm1.running_var", "features.denseblock1.denselayer1.conv1.weight", "features.denseblock1.denselayer1.norm2.weight", "features.denseblock1.denselayer1.norm2.bias", "features.denseblock1.denselayer1.norm2.running_mean", "features.denseblock1.denselayer1.norm2.running_var", "features.denseblock1.denselayer1.conv2.weight", "features.denseblock1.denselayer2.norm1.weight", "features.denseblock1.denselayer2.norm1.bias", "features.denseblock1.denselayer2.norm1.running_mean", "features.denseblock1.denselayer2.norm1.running_var", "features.denseblock1.denselayer2.conv1.weight", "features.dens